## Scrape personal finance wiki page from reddit.

In [24]:
%pip install python-dotenv
%pip install praw

  Obtaining dependency information for python-dotenv from https://files.pythonhosted.org/packages/6a/3e/b68c118422ec867fa7ab88444e1274aa40681c606d59ac27de5a5588f082/python_dotenv-1.0.1-py3-none-any.whl.metadata

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [28]:
from dotenv import load_dotenv
import os
load_dotenv()

Python-dotenv could not parse statement starting at line 3


True

In [29]:

client_id = os.getenv("REDDIT_CLIENT_ID")
client_secret = os.getenv("REDDIT_CLIENT_SECRET")
user_agent = os.getenv("REDDIT_USER_AGENT")

In [16]:
import praw
import re

# Initialize the Reddit instance with your Reddit API credentials
reddit = praw.Reddit(
    client_id = os.getenv("REDDIT_CLIENT_ID"),
    client_secret = os.getenv("REDDIT_CLIENT_SECRET"),
    user_agent = os.getenv("REDDIT_USER_AGENT")
)

subreddit_name = 'personalfinance'  # The subreddit you're interested in

def fetch_links(wiki_page_title, visited_pages=set()):
    internal_links = []
    try:
        # Fetch the wiki page content
        wiki_page = reddit.subreddit(subreddit_name).wiki[wiki_page_title]
        content = wiki_page.content_md
        # Extract all links from the content
        links = re.findall(r'\[.*?\]\((.*?)\)', content)
        # Filter out external links and prepare for recursion
        for link in links:
            if 'reddit.com/r/' + subreddit_name + '/wiki/' in link:
                internal_link = link.split('/wiki/')[-1]
                if internal_link not in visited_pages:
                    visited_pages.add(internal_link)
                    internal_links.append(link)
                    internal_links.extend(fetch_links(internal_link, visited_pages))
    except Exception as e:
        print(f"Error fetching page '{wiki_page_title}': {str(e)}")
    return list(set(internal_links))  # Return a list of unique links

# Start the process from the main wiki page or any specific page you're interested in
initial_page = 'index'  # Use 'index' for the main wiki index page
all_links = fetch_links(initial_page)
print(f"Collected {len(all_links)} unique internal wiki links:")
for link in all_links:
    print(link)

Error fetching page 'credit\_reports': INVALID_PAGE_NAME
Error fetching page 'credit\_building': INVALID_PAGE_NAME
Collected 99 unique internal wiki links:
https://www.reddit.com/r/personalfinance/wiki/credit\_building
https://www.reddit.com/r/personalfinance/wiki/readinglist
https://www.reddit.com/r/personalfinance/wiki/housing
https://www.reddit.com/r/personalfinance/wiki/employment
http://www.reddit.com/r/personalfinance/wiki/budgeting#wiki_tools.2Fsystems
https://www.reddit.com/r/personalfinance/wiki/carinsurance
https://www.reddit.com/r/personalfinance/wiki/retirementaccounts/rollovers
https://www.reddit.com/r/personalfinance/wiki/charity
https://www.reddit.com/r/personalfinance/wiki/investing#wiki_what_bond_percentage_should_i_have.3F
https://www.reddit.com/r/personalfinance/wiki/middle_aged
https://www.reddit.com/r/personalfinance/wiki/death_of_loved_one
https://www.reddit.com/r/personalfinance/wiki/index#wiki_retirement
https://www.reddit.com/r/personalfinance/wiki/country_inde

In [ ]:
%pip install requests beautifulsoup4

In [18]:
import requests
from bs4 import BeautifulSoup
import re
import time
import os

# List of URLs to extract text from
urls = all_links  # Replace <your_list_of_urls_here> with your list of URLs

def extract_relevant_text_from_url(url, retries=5, backoff_factor=1.0):
    for attempt in range(retries):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                # Logic to find the main content goes here as before
                
                main_content = soup.find('main') or soup.find('article') or soup.find('div', role='main')
                if not main_content:
                    main_content = soup.body
                text = ' '.join(t.get_text(separator='\n', strip=True) for t in main_content.find_all(recursive=False) if t.name not in ['script', 'style'])
                
                # Check if the extracted text indicates a failure to load the content properly
                if "An unknown error occurred" not in text:
                    return text
                else:
                    # If the specific error text is found, raise an exception to trigger a retry
                    raise ValueError("Content indicates an error page")
            else:
                # Non-200 responses are treated as errors and will trigger a retry
                raise ValueError(f"HTTP error {response.status_code}")
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            time.sleep(backoff_factor * (2 ** attempt))  # Exponential backoff

    return "Failed to retrieve content after several attempts"

# Function to save text to a file
def save_text_to_file(filename, text):
    # Define the base directory where files should be saved
    base_dir = "/Users/mrinoybanerjee/Desktop/Duke/Classes/Spring 2024/AIPI 540 - Deep Learning/FinanceGPT/data/pf_wiki_texts"
    # Construct the full path for the file to be saved
    full_path = os.path.join(base_dir, filename)
    with open(full_path, 'w', encoding='utf-8') as file:
        file.write(text)

# Main loop to process each URL
for url in urls:
    print(url)
    text = extract_relevant_text_from_url(url)
    # Creating a filename from the URL. Filename should be the last part of the URL
    filename = re.sub(r'\W+', '', url.split('/')[-1]) + '.txt'
    save_text_to_file(filename, text)
    print(f"Saved content from {url} to {filename}")


https://www.reddit.com/r/personalfinance/wiki/credit\_building
Attempt 1 failed: Content indicates an error page
Attempt 2 failed: Content indicates an error page
Attempt 3 failed: Content indicates an error page
Attempt 4 failed: Content indicates an error page
Attempt 5 failed: Content indicates an error page
Saved content from https://www.reddit.com/r/personalfinance/wiki/credit\_building to credit_building.txt
https://www.reddit.com/r/personalfinance/wiki/readinglist
Saved content from https://www.reddit.com/r/personalfinance/wiki/readinglist to readinglist.txt
https://www.reddit.com/r/personalfinance/wiki/housing
Attempt 1 failed: Content indicates an error page
Attempt 2 failed: Content indicates an error page
Saved content from https://www.reddit.com/r/personalfinance/wiki/housing to housing.txt
https://www.reddit.com/r/personalfinance/wiki/employment
Saved content from https://www.reddit.com/r/personalfinance/wiki/employment to employment.txt
http://www.reddit.com/r/personalfin